In [7]:
import sqlite3
import pandas as pd

con = sqlite3.connect("library.db")
cur = con.cursor()

with open("library.sql", encoding="utf-8") as f:
    cur.executescript(f.read())

# Задание 1
Вывести книги, которые были взяты в библиотеке в октябре месяце. Указать фамилии читателей, которые их взяли 
а также дату, когда их взяли. Столбцы назвать Название, Читатель, Дата соответственно. Информацию отсортирова ь
сначала по возрастанию даты, потом в алфавитном порядке по фамилиям читателей, и, наконец, по названиям к иг
тоже в алфавитном порядке.

In [13]:
query = """
    SELECT book.title as Название, reader.reader_name as Читатель, book_reader.borrow_date as Дата
    FROM book
    JOIN book_reader USING(book_id)
    JOIN reader USING(reader_id)
    WHERE strftime("%m", book_reader.borrow_date) = "10"
    ORDER BY borrow_date ASC, reader_name ASC, title ASC
"""
df = pd.read_sql(query, con)
print(df)

                    Название       Читатель        Дата
0               Смерть поэта    Иванов М.С.  2020-10-05
1             Собачье сердце    Иванов М.С.  2020-10-06
2        Одноэтажная Америка    Петров Ф.С.  2020-10-06
3      Этюд в багровых тонах    Петров Ф.С.  2020-10-06
4         Мастер и Маргарита   Самарин С.С.  2020-10-08
5   Записки о Шерлоке Холмсе   Туполев И.Д.  2020-10-08
6            Золотой теленок    Петров Ф.С.  2020-10-09
7   Записки о Шерлоке Холмсе   Федоров П.Р.  2020-10-09
8               Смерть поэта   Туполев И.Д.  2020-10-13
9                      Поэмы  Абрамова А.А.  2020-10-15
10     Пуаро ведет следствие    Иванов М.С.  2020-10-15
11        Скрюченный домишко    Иванов М.С.  2020-10-15
12        Мастер и Маргарита   Туполев И.Д.  2020-10-15
13         Пикник на обочине   Туполев И.Д.  2020-10-16
14      Убийства по алфавиту    Петров Ф.С.  2020-10-17
15         Трудно быть богом   Туполев И.Д.  2020-10-19
16     Этюд в багровых тонах    Иванов М.С.  202

# Задание 2
Определить статус каждого читателя. Вывести фамилию читателя и его статус. Если хотя бы одна книга была у читателя "на руках" более 3 недель, в столбец Статус занести "Чёрный список". Если читатель вовсе не брал пока книг в библиотеке присвоить ему статус "Неактивный читатель". Статус всех остальных читателей - "Добросовестный читатель".

In [14]:
query = """
    SELECT 
        r1.reader_name AS Читатель,
        CASE
            WHEN NOT EXISTS (SELECT * FROM book_reader br1 WHERE r1.reader_id = br1.reader_id) THEN "Неактивный читатель"
            WHEN EXISTS (SELECT * FROM book_reader br2 WHERE r1.reader_id = br2.reader_id AND 
                ((br2.return_date IS NULL AND JULIANDAY("now") - JULIANDAY(br2.borrow_date) > 21) OR 
                 (br2.return_date IS NOT NULL AND JULIANDAY(br2.return_date) - JULIANDAY(br2.borrow_date) > 21))) 
                 THEN "Чёрный список" 
            ELSE "Добросовестный читатель"
        END AS Статус
    FROM reader r1 
"""
df = pd.read_sql(query, con)
print(df)

        Читатель                   Статус
0    Иванов М.С.            Чёрный список
1    Петров Ф.С.            Чёрный список
2   Федоров П.Р.            Чёрный список
3  Абрамова А.А.  Добросовестный читатель
4   Самарин С.С.            Чёрный список
5   Туполев И.Д.            Чёрный список
6   Баранов П.В.      Неактивный читатель


# Задание 3
Найти самые популярные жанры (книги, относящиеся к которым чаще всего брали читатели) и вывести все книги, которые относятся к этим жанрам.

In [26]:
query = """
    WITH top_genre_count(count)
    AS (
        SELECT COUNT(genre.genre_name) as Взяли
        FROM genre
        JOIN book USING (genre_id)
        JOIN book_reader USING (book_id)
        GROUP BY genre.genre_name
        ORDER BY Взяли DESC
        LIMIT 1
    ), most_popular_genres(genre_id)
    AS (
        SELECT genre_id
        FROM genre, top_genre_count
        JOIN book USING (genre_id)
        JOIN book_reader USING (book_id)
        GROUP BY genre.genre_name
        HAVING COUNT(*) = top_genre_count.count
    )
    SELECT book.title AS Книга, genre.genre_name AS Жанр
    FROM book 
    JOIN genre USING (genre_id)
    WHERE book.genre_id in most_popular_genres
"""
df = pd.read_sql(query, con)
print(df)

                        Книга      Жанр
0       Пуаро ведет следствие  Детектив
1              Смерть на Ниле  Детектив
2        Убийства по алфавиту  Детектив
3     Загадочное происшествие  Детектив
4       Этюд в багровых тонах  Детектив
5  Приключения Шерлока Холмса  Детектив
6    Записки о Шерлоке Холмсе  Детектив
7          Скрюченный домишко  Детектив


# Задание 4
Читатель Самарин С.С. возвращает последнюю взятую книгу в библиотеку. Необходимо актуализировать базу данных:
    - занести текущую дату в столбец return_date соответствующей записи таблицы book_reader;
    - увеличить в таблице book на 1 количество доступных книг (available_numbers) для сдаваемой книги.
Пояснение. В запросах использовать Фамилии И.О. читателя, а не его id.

In [ ]:
reader_name = "Самарин С.С."

# Получаем id книги, взятой последней
cur.execute("""
    SELECT book_reader_id, book_id
    FROM book_reader
    JOIN reader USING (reader_id)
    WHERE reader_name = :reader_name AND book_reader.return_date IS NULL
    ORDER BY borrow_date DESC
    LIMIT 1
""", {"reader_name": reader_name})
book_reader_id, book_id = cur.fetchone()

# Заносим дату в столбец return date
cur.execute("""
    UPDATE book_reader 
    SET return_date = DATE("now")
    WHERE book_reader_id = :book_reader_id
""", {"book_reader_id": book_reader_id})

# Увеличить счётчик книги
cur.execute("""
    UPDATE book
    SET available_numbers = available_numbers + 1
    WHERE book_id = :book_id
""", {"book_id": book_id})

# Задание 5
Для тех книг, которые читатели брали больше одного раза, вывести, сколько раз брали каждую книгу,
а также минимальную разницу в днях между датами, когда читатели ее брали.
Столбцы назвать Название, Количество, Минимальный_период.
Информацию отсортировать сначала по возрастанию минимального периода, а затем по названиям книг в алфавитном порядке

Пояснение. Предположим, что одну и ту же книгу читатели взяли в следующие даты
2020-10-12, 2020-11-05, 2020-11-15. Минимальный_период вычисляется так:
    - сначала вычисляется разница между следующей и предыдущей датой
      (в нашем случае между первой и второй датой прошло 24 дня, между второй и третьей 10 дней);
    - затем находится минимальное значение (в нашем случае это 10).
Примечание. Для решения задания № 5 использовать оконные функции.

In [11]:
query = """
WITH day_differences(book_id, day_diff)
AS (
    SELECT 
        book_id, 
        JULIANDAY(borrow_date) - JULIANDAY(LAG(borrow_date) OVER (PARTITION BY book_id ORDER BY borrow_date))
    FROM book_reader
)
SELECT 
    title as Название, 
    COUNT(*) as Количество,
    MIN(day_diff) as Минимальный_период
FROM book
JOIN day_differences USING(book_id)
GROUP BY book_id
HAVING Количество >= 2
ORDER BY Минимальный_период ASC, Название ASC
"""
df = pd.read_sql(query, con)
print(df)

                    Название  Количество  Минимальный_период
0   Записки о Шерлоке Холмсе           3                 1.0
1          Трудно быть богом           3                 4.0
2        Одноэтажная Америка           3                 6.0
3         Мастер и Маргарита           2                 7.0
4               Смерть поэта           3                 8.0
5      Пуаро ведет следствие           3                 9.0
6             Смерть на Ниле           3                 9.0
7      Этюд в багровых тонах           3                14.0
8             Собачье сердце           3                15.0
9                      Поэмы           2                25.0
10        Скрюченный домишко           2                28.0
11      Убийства по алфавиту           2                29.0
12            Затерянный мир           2                30.0
13       Таинственный остров           3                32.0
14                     Поэмы           2                38.0
